In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import matplotlib.pyplot as plt


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Load and Preprocess the Dataset

In [2]:
# Load the dataset
data = pd.read_csv('xnas-itch-20230703.tbbo.csv')

# Convert price columns to correct scale
data['price'] = data['price'] / 1e9
data['bid_px_00'] = data['bid_px_00'] / 1e9
data['ask_px_00'] = data['ask_px_00'] / 1e9

# Create required columns for technical indicators
data['Close'] = data['price']
data['Volume'] = data['size']
data['High'] = data[['bid_px_00', 'ask_px_00']].max(axis=1)
data['Low'] = data[['bid_px_00', 'ask_px_00']].min(axis=1)
data['Open'] = data['Close'].shift(1).fillna(data['Close'])


# Defining the technical indicators

In [3]:

def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(series, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = series.ewm(span=fast_period, min_periods=fast_period).mean()
    slow_ema = series.ewm(span=slow_period, min_periods=slow_period).mean()
    macd = fast_ema - slow_ema
    signal = macd.ewm(span=signal_period, min_periods=signal_period).mean()
    return macd, signal, macd - signal

def calculate_stochastic(high, low, close, period=14):
    lowest_low = low.rolling(window=period).min()
    highest_high = high.rolling(window=period).max()
    k = 100 * (close - lowest_low) / (highest_high - lowest_low)
    d = k.rolling(window=3).mean()
    return k, d

def calculate_obv(close, volume):
    obv = np.where(close.diff() > 0, volume, -volume).cumsum()
    return obv

def calculate_bollinger_bands(series, period=20):
    sma = series.rolling(window=period).mean()
    std = series.rolling(window=period).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    return upper_band, sma, lower_band

def calculate_atr(high, low, close, period=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    tr = high_low.combine(high_close, np.maximum).combine(low_close, np.maximum)
    atr = tr.rolling(window=period).mean()
    return atr

def calculate_adx(high, low, close, period=14):
    plus_dm = high.diff()
    minus_dm = -low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm < 0] = 0
    tr = calculate_atr(high, low, close, period)
    plus_di = 100 * (plus_dm.ewm(span=period, min_periods=period).mean() / tr)
    minus_di = 100 * (minus_dm.ewm(span=period, min_periods=period).mean() / tr)
    dx = 100 * np.abs((plus_di - minus_di) / (plus_di + minus_di))
    adx = dx.ewm(span=period, min_periods=period).mean()
    return adx


# Calculating the Technical Indicators and Normalizing the Data 

In [4]:
# Calculate technical indicators
def calculate_technical_indicators(data):
    data['RSI'] = calculate_rsi(data['Close'])
    data['MACD'], data['MACD_signal'], _ = calculate_macd(data['Close'])
    data['Stoch_k'], data['Stoch_d'] = calculate_stochastic(data['High'], data['Low'], data['Close'])
    data['OBV'] = calculate_obv(data['Close'], data['Volume'])
    data['Upper_BB'], _, data['Lower_BB'] = calculate_bollinger_bands(data['Close'])
    data['ATR'] = calculate_atr(data['High'], data['Low'], data['Close'])
    data['ADX'] = calculate_adx(data['High'], data['Low'], data['Close'])

# Add all indicators to the dataframe
calculate_technical_indicators(data)

# Normalize the entire dataframe
scaler = MinMaxScaler()
features = ['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'Stoch_k', 'Stoch_d', 'OBV', 'Upper_BB', 'Lower_BB', 'ATR', 'ADX']
data[features] = scaler.fit_transform(data[features])


# Create a Dataset and DataLoader

In [5]:
# Create a Dataset and DataLoader
class TradeDataset(Dataset):
    def __init__(self, data, seq_len=60):
        self.data = data.dropna().reset_index(drop=True)
        self.seq_len = seq_len
        self.scaler = MinMaxScaler()
        features = ['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'Stoch_k', 'Stoch_d', 'OBV', 'Upper_BB', 'Lower_BB', 'ATR', 'ADX']
        self.data[features] = self.scaler.fit_transform(self.data[features])

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x = self.data.iloc[idx:idx+self.seq_len][['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'Stoch_k', 'Stoch_d', 'OBV', 'Upper_BB', 'Lower_BB', 'ATR', 'ADX']].values
        y = self.data.iloc[idx+self.seq_len]['Close']
        return torch.tensor(x, dtype=torch.float32).to(device), torch.tensor(y, dtype=torch.float32).to(device)

# Instantiate the dataset
dataset = TradeDataset(data)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)


# Implementing the Transformer Model

In [6]:
# Implement the Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.model_dim = model_dim
        self.input_layer = nn.Linear(input_dim, model_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.linear = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        src = self.input_layer(src)
        if src.dim() == 2:
            src = src.unsqueeze(0)  # Ensure 3D input for transformer
        src = src.permute(1, 0, 2)  # Transformer expects input shape: [seq_len, batch_size, model_dim]
        output = self.transformer_encoder(src)
        output = output.mean(dim=0)
        output = self.linear(output)
        return output


# Train the Model

In [7]:
# Initialize the model, loss function, and optimizer
model = TransformerModel(input_dim=12, model_dim=64, num_heads=8, num_layers=4, output_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model for one epoch
model.train()
epoch_loss = 0
for x_batch, y_batch in tqdm(dataloader):
    optimizer.zero_grad()
    output = model(x_batch)
    loss = criterion(output.squeeze(), y_batch)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()

print(f'Epoch 1/1, Loss: {epoch_loss/len(dataloader)}')


c:\Users\rahul\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 211/211 [04:05<00:00,  1.16s/it]

Epoch 1/1, Loss: 0.07880937649843049


# Generate Trade Signals in Batches 

In [8]:
def generate_signals_batch(model, data, batch_size=512, threshold_buy=0.005, threshold_sell=-0.005):
    model.eval()
    signals = []
    predictions = []

    def process_batch(batch, start_idx):
        x = torch.tensor(batch, dtype=torch.float32).to(device)
        if x.dim() == 2:
            x = x.unsqueeze(1)  # Ensure 3D input for transformer
        with torch.no_grad():
            preds = model(x).cpu().numpy()
        return preds, start_idx

    batches = [
        (data.iloc[i:i+batch_size][['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'Stoch_k', 'Stoch_d', 'OBV', 'Upper_BB', 'Lower_BB', 'ATR', 'ADX']].values, i)
        for i in range(0, len(data) - 60, batch_size)
    ]

    for batch, start_idx in tqdm(batches):
        preds, batch_start_idx = process_batch(batch, start_idx)
        for j, pred in enumerate(preds):
            i = batch_start_idx + j
            
            # Check if i + 60 is within bounds
            if i + 60 >= len(data):
                break
            
            if pred / data.iloc[i+60]['Close'] - 1 > threshold_buy:
                signals.append('BUY')
            elif pred / data.iloc[i+60]['Close'] - 1 < threshold_sell:
                signals.append('SELL')
            else:
                signals.append('HOLD')
            predictions.append(pred)

    return signals, predictions


# Generate and Display Predictions

In [9]:
# Generate signals and predictions
signals, predictions = generate_signals_batch(model, data)
data['Signal'] = ['HOLD'] * 60 + signals
data['Prediction'] = [None] * 60 + predictions

# Print a few rows for comparison
print(data[['Close', 'Prediction', 'Signal']].tail(20))  # Check the last 20 rows


 57%|█████▋    | 66/116 [00:04<00:03, 13.15it/s]C:\Users\rahul\AppData\Local\Temp\ipykernel_27488\1315968666.py:28: RuntimeWarning: divide by zero encountered in divide
  if pred / data.iloc[i+60]['Close'] - 1 > threshold_buy:
100%|██████████| 116/116 [00:07<00:00, 14.60it/s]

          Close    Prediction Signal
59251  0.253968  [0.19522405]   SELL
59252  0.257937  [0.27750823]    BUY
59253  0.253968  [0.24018398]   SELL
59254  0.257937  [0.23865181]   SELL
59255  0.261905  [0.38666672]    BUY
59256  0.261905   [0.2370181]   SELL
59257  0.261905  [0.24403441]   SELL
59258  0.261905  [0.32568675]    BUY
59259  0.261905  [0.32747847]    BUY
59260  0.261905  [0.24960123]   SELL
59261  0.261905  [0.18716642]   SELL
59262  0.261905  [0.18922503]   SELL
59263  0.250000  [0.15318361]   SELL
59264  0.250000   [0.2163932]   SELL
59265  0.250000   [0.2956488]    BUY
59266  0.265873  [0.28808838]    BUY
59267  0.265873  [0.29191148]    BUY
59268  0.250000  [0.29030657]    BUY
59269  0.269841  [0.28118104]    BUY
59270  0.269841  [0.28834432]    BUY
